In [32]:
import pandas as pd
import DataRetrieval
import pickle
from company_toolkit import *
from basic_analysis_toolkit import BasicAnalysis
import numpy as np
import datetime

In [33]:
d = pickle.load(open('company_dictionary.pickle', 'rb'))
sp500_dataframe = pickle.load(open('sp500_dataframe', 'rb'))
analysis = BasicAnalysis(sp500_dataframe)

In [46]:
tickers = ['AMZN', 'WM']
start = datetime.date(day=1, month=1, year=2018)
end = datetime.date(day=1, month=1, year=2019)
dict_returns = {}
for ticker in tickers:
    r = analysis.get_percent_change(ticker, start, end, 'BM')
    dict_returns.update({ticker : r})
returns_df = pd.DataFrame(dict_returns)
    
    

In [47]:
covs = returns_df.cov()
e_r = returns_df.mean()

In [63]:
def min_var_opt(sigma):
    n = len(sigma.columns)
    i = np.ones(n)
    sigma_inverse = pd.DataFrame(np.linalg.pinv(sigma.values), sigma.columns, sigma.index)
    numerator = sigma_inverse.dot(i)
    denominator = i.T.dot(sigma_inverse).dot(i)
    return numerator/denominator

def calc_var(sigma, w):
    return w.T.dot(sigma).dot(w)

def mean_var_opt(risk_param, f_e, sigma, cash):
    sigma_inverse = pd.DataFrame(np.linalg.pinv(sigma.values), sigma.columns, sigma.index)
    if cash:
        weights = (1/risk_param) * sigma_inverse.dot(f_e)
        cash_weight = 1 - weights.T.dot(np.ones(len(weights)))
        return {'Equity Weights': weights, 'Cash Weight': cash_weight}
    else:
        n = len(sigma.columns)
        i = np.ones(n)
        numer1 = sigma_inverse.dot(i)
        denom1 = i.T.dot(sigma_inverse).dot(i)
        min_var_sol = numer1/denom1
        numer2 = (i.T.dot(sigma_inverse).dot(i)) * sigma_inverse.dot(f_e) - (i.T.dot(sigma_inverse).dot(f_e)) * sigma_inverse.dot(i)
        return min_var_sol + (1/risk_param) * (numer2)/(denom1)
    
def characteristic_portfolio_weights(sigma, t):
    n = len(sigma.columns)
    i = np.ones(n)
    sigma_inverse = pd.DataFrame(np.linalg.pinv(sigma.values), sigma.columns, sigma.index)
    numer = sigma_inverse.dot(t)
    denom = t.T.dot(sigma_inverse).dot(t)
    return numer/denom


In [64]:
characteristic_portfolio_weights(covs, np.ones(2))

AMZN    0.047827
WM      0.952173
dtype: float64

In [65]:
min_var_opt(covs)

AMZN    0.047827
WM      0.952173
dtype: float64